<a href="https://colab.research.google.com/github/alex-loo/electricty_market/blob/main/usage_profile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
from sklearn.linear_model import LinearRegression


In [3]:
!pip install -- update plotly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
pd.options.display.float_format = '{:,.2f}'.format

In [10]:
file_path = "/content/drive/MyDrive/Python/AAR/Data Analysis/data"

In [106]:
df_20gw = pd.read_csv(f"{file_path}/20gw_combined.csv")
df_27pl = pd.read_csv(f"{file_path}/27pl_combined.csv")
df_8ta20 = pd.read_csv(f"{file_path}/8ta20_combined.csv")
df_103df = pd.read_csv(f"{file_path}/103df.csv")
df_11css = pd.read_csv(f"{file_path}/11css.csv")
df_30twr = pd.read_csv(f"{file_path}/30twr.csv")
df_8_10pc = pd.read_csv(f"{file_path}/8_10pc_combined.csv")

In [107]:
# Stack 103df, 11css, 30twr and 8,10PC to have the same format

df_103df = df_103df.set_index('Date').stack().reset_index()
df_103df.columns = ['Date', 'Period', 'Period_kWh']

df_11css = df_11css.set_index('Date').stack().reset_index()
df_11css.columns = ['Date', 'Period', 'Period_kWh']

df_30twr = df_30twr.set_index('Date').stack().reset_index()
df_30twr.columns = ['Date', 'Period', 'Period_kWh']

df_8_10pc = df_8_10pc.set_index('Date').stack().reset_index()
df_8_10pc.columns = ['Date', 'Period', 'Period_kWh']

In [108]:
# Creating dt objects
df_list = [df_20gw, df_27pl, df_8ta20, df_103df, df_11css, df_30twr, df_8_10pc]

for df in df_list:
  df.Date = pd.to_datetime(df.Date, format='%d/%m/%Y')
  df['Period'] = df['Period'].astype('int')
  df['Period_kWh'] = df['Period_kWh'].astype('float')

In [109]:
avg_20gw = df_20gw.groupby(by='Period', as_index=False).agg({'Period_kWh':pd.Series.mean})
avg_27pl = df_27pl.groupby(by='Period', as_index=False).agg({'Period_kWh':pd.Series.mean})
avg_8ta20 = df_8ta20.groupby(by='Period', as_index=False).agg({'Period_kWh':pd.Series.mean})
avg_103df = df_103df.groupby(by='Period', as_index=False).agg({'Period_kWh':pd.Series.mean})
avg_11css = df_11css.groupby(by='Period', as_index=False).agg({'Period_kWh':pd.Series.mean})
avg_30twr = df_30twr.groupby(by='Period', as_index=False).agg({'Period_kWh':pd.Series.mean})
avg_8_10pc = df_8_10pc.groupby(by='Period', as_index=False).agg({'Period_kWh':pd.Series.mean})

In [110]:
avg_103df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Period      48 non-null     int64  
 1   Period_kWh  48 non-null     float64
dtypes: float64(1), int64(1)
memory usage: 896.0 bytes


In [111]:
# Creating a % of daily usage column
for df in [avg_20gw, avg_27pl, avg_8ta20, avg_103df, avg_11css, avg_30twr, avg_8_10pc]:
  df['Pct_daily'] = df.Period_kWh / df.Period_kWh.sum()

In [112]:
# Compiling the avg period usage into a single Excel
with pd.ExcelWriter("/content/drive/MyDrive/Python/AAR/Data Analysis/output/output data/usage_by_period.xlsx") as writer:
    avg_20gw.to_excel(writer, sheet_name='20gw')
    avg_27pl.to_excel(writer, sheet_name='27pl')
    avg_8ta20.to_excel(writer, sheet_name='8ta20')
    avg_103df.to_excel(writer, sheet_name='103df')
    avg_11css.to_excel(writer, sheet_name='11css')
    avg_30twr.to_excel(writer, sheet_name='30twr')
    avg_8_10pc.to_excel(writer, sheet_name='8_10pc')

# Compiling the cleaned data into a single Excel
with pd.ExcelWriter("/content/drive/MyDrive/Python/AAR/Data Analysis/output/output data/data.xlsx") as writer:
    df_20gw.to_excel(writer, sheet_name='20gw')
    df_27pl.to_excel(writer, sheet_name='27pl')
    df_8ta20.to_excel(writer, sheet_name='8ta20')
    df_103df.to_excel(writer, sheet_name='103df')
    df_11css.to_excel(writer, sheet_name='11css')
    df_30twr.to_excel(writer, sheet_name='30twr')
    df_8_10pc.to_excel(writer, sheet_name='8_10pc')

In [113]:
bar_20gw = px.bar(avg_20gw,
                  x='Period',
                  y='Period_kWh',
                  color='Pct_daily',
                  color_continuous_scale='agsunset',
                  title='Typical Electricity Usage Profile for 20 Gul Way')

bar_20gw.update_traces(hovertemplate='Period: %{x} <br>Avg Consumption: %{y:.0f}kWh <br>% of Daily: %{marker.color:.2%}')
bar_20gw.update_layout(xaxis_title='Period', yaxis_title='Consumption in kWh', 
                  coloraxis_showscale=False)
bar_20gw.update_xaxes(nticks=48)
bar_20gw.write_html("/content/drive/MyDrive/Python/AAR/Data Analysis/output/charts/20gw_usage_profile.html")
bar_20gw.show()

In [114]:
bar_27pl = px.bar(avg_27pl,
                    x='Period',
                    y='Period_kWh',
                    color='Pct_daily',
                    color_continuous_scale='agsunset',
                    title='Typical Electricity Usage Profile for 27 Penjuru Lane')

bar_27pl.update_traces(hovertemplate='Period: %{x} <br>Avg Consumption: %{y:.0f}kWh <br>% of Daily: %{marker.color:.2%}')
bar_27pl.update_layout(xaxis_title='Period', yaxis_title='Consumption in kWh', 
                  coloraxis_showscale=False)
bar_27pl.update_xaxes(nticks=48)
bar_27pl.write_html("/content/drive/MyDrive/Python/AAR/Data Analysis/output/charts/27pl_usage_profile.html")
bar_27pl.show()

In [115]:
bar_8ta = px.bar(avg_8ta20,
                  x='Period',
                  y='Period_kWh',
                  color='Pct_daily',
                  color_continuous_scale='agsunset',
                  title='Typical Electricity Usage Profile for 8 Tuas Ave 20')

bar_8ta.update_traces(hovertemplate='Period: %{x} <br>Avg Consumption: %{y:.0f}kWh <br>% of Daily: %{marker.color:.2%}')
bar_8ta.update_layout(xaxis_title='Period', yaxis_title='Consumption in kWh', 
                  coloraxis_showscale=False)
bar_8ta.update_xaxes(nticks=48)
bar_8ta.write_html("/content/drive/MyDrive/Python/AAR/Data Analysis/output/charts/8ta20_usage_profile.html")
bar_8ta.show()

In [116]:
bar_103df = px.bar(avg_103df,
                    x='Period',
                    y='Period_kWh',
                    color='Pct_daily',
                    color_continuous_scale='agsunset',
                    title='Typical Electricity Usage Profile for 103 Defu')

bar_103df.update_traces(hovertemplate='Period: %{x} <br>Avg Consumption: %{y:.0f}kWh <br>% of Daily: %{marker.color:.2%}')
bar_103df.update_layout(xaxis_title='Period', yaxis_title='Consumption in kWh', 
                  coloraxis_showscale=False)
bar_103df.update_xaxes(nticks=48)
bar_103df.write_html("/content/drive/MyDrive/Python/AAR/Data Analysis/output/charts/103df_usage_profile.html")
bar_103df.show()

In [118]:
bar_11css = px.bar(avg_11css,
                    x='Period',
                    y='Period_kWh',
                    color='Pct_daily',
                    color_continuous_scale='agsunset',
                    title='Typical Electricity Usage Profile for 11 Changi South')

bar_11css.update_traces(hovertemplate='Period: %{x} <br>Avg Consumption: %{y:.0f}kWh <br>% of Daily: %{marker.color:.2%}')
bar_11css.update_layout(xaxis_title='Period', yaxis_title='Consumption in kWh', 
                  coloraxis_showscale=False)
bar_11css.update_xaxes(nticks=48)
bar_11css.write_html("/content/drive/MyDrive/Python/AAR/Data Analysis/output/charts/11css_usage_profile.html")
bar_11css.show()

In [119]:
bar_30twr = px.bar(avg_30twr,
                    x='Period',
                    y='Period_kWh',
                    color='Pct_daily',
                    color_continuous_scale='agsunset',
                    title='Typical Electricity Usage Profile for 30 Tuas West Road')

bar_30twr.update_traces(hovertemplate='Period: %{x} <br>Avg Consumption: %{y:.0f}kWh <br>% of Daily: %{marker.color:.2%}')
bar_30twr.update_layout(xaxis_title='Period', yaxis_title='Consumption in kWh', 
                  coloraxis_showscale=False)
bar_30twr.update_xaxes(nticks=48)
bar_30twr.write_html("/content/drive/MyDrive/Python/AAR/Data Analysis/output/charts/30twr_usage_profile.html")
bar_30twr.show()

In [120]:
bar_8_10pc = px.bar(avg_8_10pc,
                    x='Period',
                    y='Period_kWh',
                    color='Pct_daily',
                    color_continuous_scale='agsunset',
                    title='Typical Electricity Usage Profile for 8 & 10 Pandan Crescent')

bar_8_10pc.update_traces(hovertemplate='Period: %{x} <br>Avg Consumption: %{y:.0f}kWh <br>% of Daily: %{marker.color:.2%}')
bar_8_10pc.update_layout(xaxis_title='Period', yaxis_title='Consumption in kWh', 
                  coloraxis_showscale=False)
bar_8_10pc.update_xaxes(nticks=48)
bar_8_10pc.write_html("/content/drive/MyDrive/Python/AAR/Data Analysis/output/charts/8_10pc_usage_profile.html")
bar_8_10pc.show()